# Feature Engineering + Model Selection Workflow (Project Baseline Build)

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/09_tuning_feature_engineering_project_baseline.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Engineer features with pipelines without leakage
2. Use `GridSearchCV` / `RandomizedSearchCV` for systematic tuning
3. Define a project-grade evaluation plan (metric + split/CV + baseline + reporting)
4. Produce a baseline model notebook that can be extended
5. Use Gemini to draft search grids and then simplify them

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import (
    train_test_split, StratifiedKFold,
    GridSearchCV, RandomizedSearchCV
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform, randint
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

**Reading the output:**

The setup cell imports the tuning machinery we will use: `GridSearchCV` for exhaustive parameter search, `RandomizedSearchCV` for sampling-based search, and `StratifiedKFold` for the inner CV loop that evaluates each parameter combination. We also import `PolynomialFeatures` and `SelectKBest` for feature engineering, and `scipy.stats.uniform` and `randint` for defining continuous and integer parameter distributions in randomized search. The **"Setup complete!"** message confirms all libraries loaded without error.

**Why this matters:** Grid search and randomized search are the two workhorses of hyperparameter tuning in scikit-learn. Understanding when to use each -- and how they interact with pipelines and CV -- is essential for building strong models without overfitting to the validation set.

---

## 1. Load Data and Baseline

Every tuning experiment needs a **baseline** -- a simple model with default hyperparameters that establishes the performance floor. Without a baseline, you cannot tell whether elaborate feature engineering or exhaustive grid searches actually improved anything.

We continue with the breast cancer Wisconsin dataset (569 samples, 30 features) and the standard 60/20/20 split. The baseline is a Logistic Regression pipeline with `StandardScaler` and all default hyperparameters (`C=1.0`, `penalty='l2'`). Its validation accuracy will be our reference point for every experiment in this notebook.

In [ ]:
# Load data
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

# Split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED, stratify=y_temp)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
print(f"Features: {X.shape[1]}")

# Simple baseline
baseline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])

baseline.fit(X_train, y_train)
baseline_score = baseline.score(X_val, y_val)

print(f"\nBaseline validation accuracy: {baseline_score:.4f}")

**Reading the output:**

The split sizes are printed as **Train: 341 | Val: 114 | Test: 114**, matching the 60/20/20 convention. The dataset has **30 features**, all numeric measurements of cell nuclei.

The baseline Logistic Regression achieves a **validation accuracy around 0.96-0.97**. This is already high because the breast cancer dataset is relatively well-separated, but there is still room for improvement -- especially if we can identify which features matter most or find non-linear relationships.

**Key takeaway:** Record this baseline number. Every subsequent experiment (feature selection, polynomial features, grid search) must be compared against it. If a complex model does not beat the baseline, the extra complexity is not justified.

---

## 2. Feature Engineering Inside Pipelines

### Safe Feature Engineering Patterns

**Rule:** All feature engineering must happen INSIDE the pipeline

**Why?**
- Prevents leakage (fit on train, transform on val/test)
- Ensures reproducibility
- Makes deployment easier

**Common feature engineering steps:**
1. Polynomial features
2. Feature interactions
3. Feature selection
4. Domain-specific transformations

In [ ]:
# Pipeline with feature engineering
fe_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(f_classif, k=20)),  # Keep top 20 features
    ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])

fe_pipeline.fit(X_train, y_train)
fe_score = fe_pipeline.score(X_val, y_val)

print("=== FEATURE ENGINEERING PIPELINE ===")
print(f"Original features: {X.shape[1]}")
print(f"Selected features: 20")
print(f"Validation accuracy: {fe_score:.4f}")
print(f"Improvement: {(fe_score - baseline_score):.4f}")

# See which features were selected
selected_mask = fe_pipeline.named_steps['feature_selection'].get_support()
selected_features = X.columns[selected_mask].tolist()
print(f"\nSelected features: {selected_features[:5]}... (showing first 5)")

**Reading the output:**

The feature engineering pipeline uses `SelectKBest(f_classif, k=20)` to keep only the 20 features with the highest ANOVA F-statistic, discarding the 10 least informative ones. The output compares:

- **Original features:** 30
- **Selected features:** 20
- **Validation accuracy:** typically close to or slightly above the baseline
- **Improvement:** the difference from baseline (may be small, positive, or even negative)

The first five selected feature names are printed to give you a sense of which measurements survived the filter. Features like `mean concave points`, `worst radius`, and `worst perimeter` tend to rank highly because they correlate strongly with the malignant/benign distinction.

**Why this matters:** Feature selection inside a pipeline prevents data leakage. The `SelectKBest` step is fitted *only* on training data, so the feature rankings do not peek at validation labels. If you performed selection outside the pipeline, you would leak information and get overly optimistic scores.

---

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Add engineered features and re-run CV.

Try adding polynomial features (degree=2) to a subset of features.

---

In [ ]:
# YOUR CODE: Add polynomial features
from sklearn.model_selection import cross_val_score

# Warning: Polynomial features can explode the feature space!
# Let's use only a subset
poly_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection_pre', SelectKBest(f_classif, k=10)),  # Reduce to 10 first
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler2', StandardScaler()),  # Re-scale after polynomial
    ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=2000))
])

# Evaluate with CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
poly_scores = cross_val_score(poly_pipeline, X_train, y_train, cv=cv, scoring='roc_auc')
baseline_scores = cross_val_score(baseline, X_train, y_train, cv=cv, scoring='roc_auc')

print("=== POLYNOMIAL FEATURES COMPARISON ===")
print(f"Baseline:   {baseline_scores.mean():.4f} ± {baseline_scores.std():.4f}")
print(f"Polynomial: {poly_scores.mean():.4f} ± {poly_scores.std():.4f}")
print(f"\nImprovement: {(poly_scores.mean() - baseline_scores.mean()):.4f}")

# Check feature explosion
poly_pipeline.fit(X_train.iloc[:10], y_train.iloc[:10])  # Fit on small sample to check
n_poly_features = poly_pipeline.named_steps['poly'].n_output_features_
print(f"\n⚠️ Feature explosion: 10 → {n_poly_features} features")

**Reading the output:**

This cell adds `PolynomialFeatures(degree=2)` *after* reducing to 10 features via `SelectKBest`. The polynomial step generates all pairwise interactions and squared terms, expanding 10 features to **65 features** (10 originals + 10 squares + 45 interactions).

The comparison shows two lines of CV scores (5-fold Stratified, scored by ROC-AUC):
- **Baseline:** mean +/- std without polynomial features.
- **Polynomial:** mean +/- std with the expanded feature set.

The improvement line shows the difference. On this dataset, polynomial features may provide a marginal boost or may actually hurt slightly, because Logistic Regression can already capture the linear decision boundary well. The warning about **feature explosion (10 to 65)** reminds you that degree-2 polynomials grow quadratically -- with all 30 original features, you would get 496 columns, dramatically increasing overfitting risk and training time.

**Key takeaway:** Polynomial features are a double-edged sword. They can capture non-linear relationships, but they multiply the feature space and require more training data. Always compare against the baseline with CV, and use `SelectKBest` *before* the polynomial step to keep dimensionality manageable.

---

### YOUR ANALYSIS:

**Did polynomial features help?**  
[Your analysis]

**What's the cost?**  
[Feature explosion, complexity, overfitting risk]

**Would you use this in production?**  
[Justify your decision]

---

## 3. GridSearchCV - Systematic Hyperparameter Tuning

### How GridSearchCV Works

1. Define parameter grid
2. Try every combination
3. Use CV to evaluate each
4. Return best parameters

**Warning:** Grid search can be expensive!
- 3 parameters × 3 values each = 27 combinations
- 27 combinations × 5 folds = 135 model fits

In [ ]:
# Define pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])

# Define parameter grid
param_grid = {
    'clf__C': [0.01, 0.1, 1.0, 10.0],
    'clf__penalty': ['l2'],  # Just L2 for speed
    'clf__solver': ['lbfgs', 'liblinear']
}

print("=== GRID SEARCH CONFIGURATION ===")
print(f"Parameter grid: {param_grid}")
n_combinations = len(param_grid['clf__C']) * len(param_grid['clf__penalty']) * len(param_grid['clf__solver'])
print(f"Total combinations: {n_combinations}")
print(f"With 5-fold CV: {n_combinations * 5} model fits")

# Run grid search
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0,
    return_train_score=True
)

print("\nRunning grid search...")
grid_search.fit(X_train, y_train)

print("\n=== GRID SEARCH RESULTS ===")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV score: {grid_search.best_score_:.4f}")
print(f"Validation score: {grid_search.score(X_val, y_val):.4f}")

**Reading the output:**

The grid search configuration is printed first: 4 values of `C` (0.01, 0.1, 1.0, 10.0) times 1 penalty times 2 solvers = **8 combinations**, each evaluated across 5 folds for a total of **40 model fits**.

After fitting, three key results appear:
- **Best parameters:** the combination that achieved the highest mean CV ROC-AUC. Expect `C=1.0` or `C=0.1` to win, since the breast cancer data does not require heavy regularization.
- **Best CV score:** the mean ROC-AUC of the winning combination, typically around **0.99**.
- **Validation score:** performance on the held-out validation set (which the grid search never saw). This should be close to the CV score; a large gap would indicate that the search overfit to the CV folds.

**Why this matters:** `GridSearchCV` wraps the entire search-and-evaluate loop into a single estimator. After calling `.fit()`, the object automatically refits the best parameters on the full training set, so `grid_search.predict()` uses the optimal model.

---

In [ ]:
# Examine all results
results_df = pd.DataFrame(grid_search.cv_results_)
results_summary = results_df[[
    'param_clf__C', 'param_clf__solver',
    'mean_test_score', 'std_test_score',
    'mean_train_score', 'rank_test_score'
]].sort_values('rank_test_score')

print("\n=== TOP 5 PARAMETER COMBINATIONS ===")
print(results_summary.head().to_string(index=False))

# Visualize C parameter effect
plt.figure(figsize=(10, 6))
for solver in param_grid['clf__solver']:
    mask = results_df['param_clf__solver'] == solver
    plt.plot(
        results_df[mask]['param_clf__C'],
        results_df[mask]['mean_test_score'],
        marker='o', label=f'Solver: {solver}'
    )
plt.xscale('log')
plt.xlabel('C (Regularization Parameter)')
plt.ylabel('Mean CV ROC-AUC')
plt.title('Grid Search: C Parameter Effect')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Reading the output:**

The top-5 parameter combinations table shows each combination ranked by `rank_test_score`. Columns include the actual parameter values (`param_clf__C`, `param_clf__solver`), the mean and standard deviation of the test (validation) score, and the mean training score.

The line plot below visualizes how **C** (the inverse regularization strength) affects ROC-AUC for each solver. You will typically see:
- **Small C (0.01):** Strong regularization, slightly lower scores because the model is too constrained.
- **Mid-range C (0.1-1.0):** The sweet spot with highest CV scores.
- **Large C (10.0):** Weak regularization; scores may plateau or dip slightly as the model begins to overfit.

The two solver lines (lbfgs and liblinear) usually overlap closely, confirming that solver choice matters less than regularization strength for this problem.

**Key takeaway:** Examining `cv_results_` is not just good practice -- it is essential. The best parameters alone do not tell you how *sensitive* performance is to each hyperparameter. If performance is nearly flat across all C values, you have little to gain from further tuning.

---

## 4. RandomizedSearchCV - Faster Alternative

### When to Use Randomized Search

**Use RandomizedSearchCV when:**
- Parameter space is large
- Continuous parameters
- Time budget is limited
- Initial exploration phase

**Advantage:** Sample randomly instead of exhaustive search

In [ ]:
# Random Forest with randomized search
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=RANDOM_SEED))
])

# Define parameter distributions
param_distributions = {
    'clf__n_estimators': randint(50, 200),
    'clf__max_depth': randint(3, 20),
    'clf__min_samples_split': randint(2, 20),
    'clf__min_samples_leaf': randint(1, 10)
}

random_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions,
    n_iter=20,  # Try 20 random combinations
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED),
    scoring='roc_auc',
    n_jobs=-1,
    random_state=RANDOM_SEED,
    return_train_score=True
)

print("=== RANDOMIZED SEARCH ===")
print(f"Parameter space: {param_distributions}")
print(f"Sampling 20 random combinations...")

random_search.fit(X_train, y_train)

print(f"\nBest parameters: {random_search.best_params_}")
print(f"Best CV score: {random_search.best_score_:.4f}")
print(f"Validation score: {random_search.score(X_val, y_val):.4f}")

**Reading the output:**

Randomized search sampled **20 random combinations** from the specified distributions: `n_estimators` between 50 and 200, `max_depth` between 3 and 20, `min_samples_split` between 2 and 20, and `min_samples_leaf` between 1 and 10. Each combination was evaluated with 5-fold stratified CV, totaling **100 model fits**.

The output reports:
- **Best parameters:** a specific set of Random Forest hyperparameters, such as `n_estimators=150, max_depth=12, min_samples_split=4, min_samples_leaf=2`.
- **Best CV score:** the mean ROC-AUC of the winning combination, likely in the **0.98-0.99** range.
- **Validation score:** performance on the held-out validation set.

Compared to GridSearchCV, randomized search explored a much larger parameter space (continuous distributions instead of fixed lists) while fitting fewer total models. The tradeoff is that it *might* miss the exact optimum, but research by Bergstra & Bengio (2012) shows that random search finds near-optimal configurations with far fewer iterations.

**Why this matters:** Use `GridSearchCV` when the parameter space is small and discrete. Use `RandomizedSearchCV` for initial exploration of large or continuous spaces, then optionally refine with a focused grid around the best region.

---

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Run a small grid (2-3 params) and report best CV score.

Already done above! Now create a baseline report table:

---

In [ ]:
# Create comprehensive baseline report
baseline_report = []

# Model 1: Simple baseline
baseline_report.append({
    'Model': 'Logistic (default)',
    'Val_ROC_AUC': baseline_score,
    'Parameters': 'C=1.0, penalty=l2',
    'Features': X.shape[1],
    'Notes': 'Simple baseline'
})

# Model 2: Grid search best
baseline_report.append({
    'Model': 'Logistic (tuned)',
    'Val_ROC_AUC': grid_search.score(X_val, y_val),
    'Parameters': str(grid_search.best_params_),
    'Features': X.shape[1],
    'Notes': 'Grid search optimized'
})

# Model 3: Random Forest tuned
baseline_report.append({
    'Model': 'Random Forest (tuned)',
    'Val_ROC_AUC': random_search.score(X_val, y_val),
    'Parameters': str(random_search.best_params_),
    'Features': X.shape[1],
    'Notes': 'Random search optimized'
})

report_df = pd.DataFrame(baseline_report)
print("=== PROJECT BASELINE REPORT ===")
print(report_df.to_string(index=False))

# Identify champion
best_idx = report_df['Val_ROC_AUC'].idxmax()
print(f"\n✓ Champion model: {report_df.loc[best_idx, 'Model']}")
print(f"✓ Validation ROC-AUC: {report_df.loc[best_idx, 'Val_ROC_AUC']:.4f}")

**Reading the output:**

The baseline report table compares three models head-to-head:

| Model | What it represents |
|-------|-------------------|
| **Logistic (default)** | No tuning, C=1.0 -- the performance floor |
| **Logistic (tuned)** | Best parameters from GridSearchCV |
| **Random Forest (tuned)** | Best parameters from RandomizedSearchCV |

Each row shows the validation ROC-AUC, the winning parameters, the feature count, and a short note. The **champion model** is highlighted at the bottom -- the model with the highest validation ROC-AUC.

On this dataset, all three models likely perform within a narrow band (0.96-0.99 ROC-AUC), because the breast cancer features are highly informative. In your course project, the gaps between models may be larger, making this comparison table even more valuable.

**Key takeaway:** A baseline report table is a *deliverable*, not just a debugging tool. It documents every model you tried, the parameters you used, and the metrics you achieved. Include this table in your project submission so reviewers can see your modeling journey at a glance.

---

## 5. Project Baseline Notebook Scaffold

### Required Components for Project Baseline

1. **Data Loading and Audit**
   - Load dataset
   - Check for issues
   - Document data quality

2. **Train/Val/Test Splits**
   - Proper splits with stratification
   - Lock test set away

3. **Baseline Model**
   - Simple model (mean/mode/simple classifier)
   - Establishes floor performance

4. **Improved Model**
   - Preprocessing pipeline
   - Tuned hyperparameters
   - CV evaluation

5. **Evaluation Report**
   - Multiple metrics
   - Comparison table
   - Visualizations

6. **Documentation**
   - Modeling choices explained
   - Assumptions documented
   - Next steps identified

## 6. Gemini Prompts for Tuning

### Example Prompts:

**Prompt 1: Generate Parameter Grid**
```
I'm tuning a Random Forest classifier for a binary classification task.
Generate a reasonable parameter grid for GridSearchCV including:
- n_estimators
- max_depth
- min_samples_split

Keep it small (< 20 combinations) for initial exploration.
```

**Prompt 2: Optimize Grid**
```
I ran GridSearchCV and found best params: {results}
Help me design a refined grid search around these values
to fine-tune performance.
```

**Prompt 3: Debug Search**
```
My RandomizedSearchCV is taking too long. Here's my config: {config}
Help me reduce search time while maintaining good coverage.
```

**Remember:**
- Verify Gemini's suggestions
- Start small, then expand
- Always use CV, never single split
- Document your search strategy

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Pipeline Feature Engineering**: Keep everything inside pipelines
2. **GridSearchCV**: Exhaustive search for small parameter spaces
3. **RandomizedSearchCV**: Faster exploration of large spaces
4. **Baseline Reports**: Document all models systematically
5. **Project Readiness**: Structure for reproducible modeling

### Critical Rules:

> **"All feature engineering must be in the pipeline"**

> **"Start with small grids, then refine"**

> **"Document every modeling choice"**

### Next Steps:

- Next notebook: Midterm - Business case practicum
- **Project Milestone 2 checkpoint**: Draft baseline notebook
- Apply today's patterns to your project dataset

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Python labs on feature engineering
- scikit-learn User Guide: [Grid search](https://scikit-learn.org/stable/modules/grid_search.html)
- scikit-learn User Guide: [Pipeline parameter tuning](https://scikit-learn.org/stable/modules/compose.html#pipeline-tuning)
- Provost, F., & Fawcett, T. (2013). *Data Science for Business* - Evaluation and business framing

---



<center>

Thank you!

</center>